### Nedostajuće vrednosti

Neretko u skupovima podataka kojimo raspolažemo postoje nedostajuće vrednosti. U `Python` bibliotekama nedostajućim vrednostima se smatraju `None` i `NaN` vrednosti. Ovaj skup vrednosti se može proširiti i svim vrednostima koje nisu iz zadovoljavajućih opsega ili kategorija za posmatrani problem. Na primer, redni broj meseca ne može imati vrednost 13 ili prosečan broj pacijenata jedne ordinacije ne može biti negativan. U tom smislu bi se ove vrednosti mogle zameniti nedostajućim. 

Cilj narednih primera je da približe rad sa funkcijama koje omogućavaju manipulaciju nedostajućim vrednostima, kao i da uvedu primene linearne regresije u oceni nedostajućih vrednosti. 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import metrics

Prvo generisemo proizvoljni skup tacaka.

In [3]:
np.random.seed(7)

In [4]:
N = 10
x = np.random.randn(N)
y = np.random.rand(N)

In [5]:
points = pd.DataFrame({'x':x, 'y': y})

In [6]:
# brz uvid u brojnost atributa i njihove vrednosti
points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
x    10 non-null float64
y    10 non-null float64
dtypes: float64(2)
memory usage: 288.0 bytes


In [7]:
# proglasavamo par vrednosti nedostajucima
points.loc[5, 'y'] = None
points.loc[6, 'y'] = np.nan
points.loc[9,'x'] = np.nan
points.loc[9,'y'] = np.nan

In [8]:
points

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,NaN
6,-0.000890,NaN
7,-1.754724,0.024899
8,1.017658,0.600549
9,NaN,NaN


In [9]:
points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
x    9 non-null float64
y    7 non-null float64
dtypes: float64(2)
memory usage: 288.0 bytes


Funkcija `isna` Pandas biblioteke proverava koje vrednosti su nedostajuće. Ravnopravno se može koristiti i funkcija `isnull`.

In [10]:
points.isna()

,x,y
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,True
6,False,True
7,False,False
8,False,False
9,True,True


Funkcija `notna` Pandas biblioteke proverava koje vrednosti nisu nedostajuće. Ravnopravno se može koristiti i funkcija `notnull`.

In [11]:
points.notna()

,x,y
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,False
6,True,False
7,True,True
8,True,True
9,False,False


Funkcija `isnan` numpy biblioteke proverava da li je vrednost nedostajuća. 

In [12]:
np.isnan(points.loc[5, 'x'])

False

<div class="alert alert-success">
Neke od opcija koje imamo na raspolaganju ukoliko su nedostajuće vrednosti prisutne u skupu podataka: 
</div>

1. brisanje nedostajućih vrednosti 

Moze se koristi `dropna` funkcija Pandas biblioteke.

In [13]:
# ovaj poziv ce obrisati 9. tacku
points.dropna(axis = 0, how = 'all', inplace = True)

# ovaj poziv ce obrisati 5. i 6. tacku
# points.dropna(axis = 0, how = 'any', inplace = True)

points

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,NaN
6,-0.000890,NaN
7,-1.754724,0.024899
8,1.017658,0.600549


2. zamena nedostajuće vrednosti prosečnom vrednošću, medijanom ili modom

In [14]:
y_mean = points[points['y'].notna()]['y'].mean()
y_mean

0.3547785865252053

In [15]:
points['y'].replace(to_replace=np.nan, value = y_mean, inplace=True)

In [16]:
points

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,0.354779
6,-0.000890,0.354779
7,-1.754724,0.024899
8,1.017658,0.600549


3. zamena nedostajućih vrednosti nekim proizvoljnim vrednostima iz skupa vrednosti

In [17]:
def random_imputation(data, feature):
    number_of_missing_values = data[feature].isna().sum()
    observed_values = data.loc[data[feature].notna(), feature]
    
    new_values = np.random.choice(observed_values, number_of_missing_values)
    
    data.loc[data[feature].isna(), feature] = new_values
    
    return data

In [18]:
random_imputation(points, 'y')

,x,y
0,1.690526,0.380941
1,-0.465937,0.065936
2,0.032820,0.288146
3,0.407516,0.909594
4,-0.788923,0.213385
5,0.002066,0.354779
6,-0.000890,0.354779
7,-1.754724,0.024899
8,1.017658,0.600549


4. aproksimiranje vrednosti na osnovu vrednosti nekih suseda

<div class="alert alert-success">
Zadatak
</div>

U primeru koji sledi bi hteli da na osnovu poznatih vrednosti atributa korišćenjem linearne regresije aproksimiramo vrednosti koje nedostaju. Skup podataka sa kojim ćemo raditi je preuzet sa [adrese](https://www.kaggle.com/ranjeetjain3/seaborn-tips-dataset) i sadrži informacije o napojnicama zaposlenih u jednom restoranu. Osim visine napojnice poznati su i iznos ukupnog računa, informacija o kom obroku se radilo, danu u nedelji, veličini grupe, polu osobe koja je platila i sekciji restorana za pušače ili nepušače. 

In [19]:
data = pd.read_csv('data/tips.csv')

In [20]:
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float64
tip           244 non-null float64
sex           244 non-null object
smoker        244 non-null object
day           244 non-null object
time          244 non-null object
size          244 non-null int64
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [22]:
data.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [23]:
N = data.shape[0]
N

244

Prvo ćemo pripremiti skup podataka tako što ćemo sve kategoričke atribute zameniti odgovarajućim numeričkim ili kombinacijom numeričkih. 

I način zamene vrednosti: korišćenje `replace` funkcije Pandas biblioteke

In [24]:
data['smoker'].unique()

array(['No', 'Yes'], dtype=object)

In [25]:
data['smoker'].replace({'No': 0, 'Yes':1}, inplace=True)

In [26]:
data['sex'].unique()

array(['Female', 'Male'], dtype=object)

In [27]:
data['sex'].replace({'Female': 0, 'Male':1}, inplace = True)

II način zamene vrednosti: korišćenje `get_dummies` funkcije Pandas biblioteke
<br>
<br>
Ovaj način je podesniji ukoliko atribut uzima više vrednosti. 

Funkcijom `get_dummies` se vrši takozvano `one-hot` kodiranje kojim se odgovarajućoj vrednosti pridružuje vektor svih nula i jedne jedinice čija pozicija u vektoru odgovara poziciji te vrednosti u skupu vrednosti. Na primer, ako su vrednosti u skupu Sun, Sat, Thur i Fri, njima pridruženi vektori su (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) i (0, 0, 0, 1). 

Ukoliko se raspolaže skupom od `k` vrednosti, obično se na ovaj način kodira `k-1` vrednosti jer se vektor nula indirektno veže za nerazmatranu vrednost. Ovaj pristup je dobar zbog kontrolisane redudantnosti i korelisanosti dobijenih atributa. U funkciji `get_dummies` to dozvoljava parametar `drop_first`.

In [28]:
data['day'].unique()

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [29]:
day = pd.get_dummies(data['day'], prefix = 'day', drop_first=True)

In [30]:
data['time'].unique()

array(['Dinner', 'Lunch'], dtype=object)

In [31]:
time = pd.get_dummies(data['time'], prefix = 'time', drop_first=True)

In [32]:
# kolone vrednosti dobijene na prethodni nacin se nadovezuju na polazni skup podataka
data = pd.concat([data, day, time], axis = 1)

In [33]:
# dok se originalne kolone day i time brisu
data.drop(['day','time'], axis = 1, inplace= True)

In [34]:
data.head()

,total_bill,tip,sex,smoker,size,day_Sat,day_Sun,day_Thur,time_Lunch
0,16.99,1.01,0,0,2,0,1,0,0
1,10.34,1.66,1,0,3,0,1,0,0
2,21.01,3.50,1,0,3,0,1,0,0
3,23.68,3.31,1,0,2,0,1,0,0
4,24.59,3.61,0,0,4,0,1,0,0


Neke vrednosti napojnica ćemo veštački proglasiti nedostajućim. Vrednosti ćemo pre brisanja sačuvati zbog evaluacije. Za predikciju ćemo koristiti vrednosti atributa total_bill, size i smoker. 

In [35]:
mask_indexes = np.random.randint(0, N, 50)

In [36]:
test = data.loc[mask_indexes]
y_test = test['tip']
X_test = test.drop(['tip'], axis= 1)
X_test = X_test[['total_bill', 'size', 'smoker']]

In [37]:
train = data.drop(mask_indexes, axis = 0)
y_train = train['tip']
X_train = train.drop(['tip'], axis = 1)
X_train = X_train[['total_bill', 'size', 'smoker']]

In [38]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
y_predicted = model.predict(X_test)

In [41]:
metrics.r2_score(y_test, y_predicted)

0.506006430672497

Ovaj pristup aproksimacije nedostajućih vrednosti se često koristi u sistemima preporuka za nove proizvode u rešavanju takozvanog hladnog starta (eng. cold start) gde zbog nepoznavanja mišljenja ili ocena korisnika proizvod ne može da se probije na listi preporuka.